# A Novel Embedding Model for Knowledge Base Completion Based on Convolutional Neural Network

## Abstract
Trong bài báo này, chúng tôi đề xuất một mô hình nhúng mới, có tên ConvKB, để hoàn thành cơ sở tri thức. Mô hình ConvKB của chúng tôi cải tiến các mô hình tiên tiến bằng cách sử dụng mạng thần kinh tích chập, để nó có thể nắm bắt các mối quan hệ toàn cầu và các đặc điểm chuyển tiếp giữa các thực thể và quan hệ trong các cơ sở tri thức. Trong ConvKB, mỗi bộ ba (thực thể đầu, quan hệ, thực thể đuôi) được biểu diễn dưới dạng ma trận 3 cột trong đó mỗi vectơ cột biểu thị một phần tử ba. Ma trận 3 cột này sau đó được đưa đến một lớp chập trong đó nhiều bộ lọc được vận hành trên ma trận để tạo ra các bản đồ tính năng khác nhau. Các bản đồ tính năng này sau đó được nối vào một vectơ tính năng duy nhất đại diện cho bộ ba đầu vào. Vectơ đặc trưng được nhân với một vectơ trọng lượng thông qua một sản phẩm chấm để trả về điểm số. Điểm này sau đó được sử dụng để dự đoán liệu bộ ba có hợp lệ hay không. Các thử nghiệm cho thấy ConvKB đạt được hiệu suất dự đoán liên kết tốt hơn so với các mô hình nhúng tiên tiến trước đây trên hai bộ dữ liệu điểm chuẩn WN18RR và FB15k-237

## Introduction

Các cơ sở tri thức quy mô lớn (KB), như YAGO (suchanek et al., 2007), Freebase (Bollacker et al., 2008) và DBpedia (Lehmann et al., 2015), thường là cơ sở dữ liệu của bộ ba đại diện cho các mối quan hệ giữa các thực thể ở dạng thực tế (thực thể đầu, quan hệ, thực thể đuôi) được ký hiệu là (h, r, t), ví dụ: (Melbourne, cityOf, Úc). Các KB này là tài nguyên hữu ích trong nhiều ứng dụng như tìm kiếm và xếp hạng ngữ nghĩa (Kasneci et al., 2008; Schuhmacher và Ponzetto, 2014; Xiong et al., 2017), trả lời câu hỏi (Zhang et al., 2016; Hao et al. , 2017) và đọc máy (Yang và Mitchell, 2017). Tuy nhiên, KB vẫn chưa hoàn thành, tức là, thiếu rất nhiều bộ ba hợp lệ (Socher et al., 2013; West et al., 2014). Do đó, nhiều công việc nghiên cứu đã được dành cho việc hoàn thành cơ sở tri thức hoặc dự đoán liên kết để dự đoán liệu một bộ ba (h, r, t) có hợp lệ hay không (Bordes et al., 2011).

Nhiều mô hình nhúng đã đề xuất để tìm hiểu các biểu diễn vectơ hoặc ma trận cho các thực thể và quan hệ, thu được kết quả dự đoán liên kết hiện đại (SOTA) (Niken và cộng sự, 2016a). Trong các mô hình nhúng này, bộ ba hợp lệ có được điểm số thấp hơn so với bộ ba không hợp lệ. Chúng ta hãy lấy mô hình nhúng nổi tiếng TransE (Bordes et al., 2013) làm ví dụ. Trong TransE, các thực thể và quan hệ được thể hiện bằng các nhúng vector kdimensional. TransE sử dụng một đặc tính chuyển tiếp để mô hình hóa các mối quan hệ giữa các thực thể, trong đó nó giả định rằng nếu (h, r, t) là một thực tế hợp lệ, thì việc nhúng thực thể đầu h cộng với việc nhúng quan hệ r phải gần với việc nhúng thực thể đuôi t, tức là vh + vr ≈ vt (ở đây, vh, vr và vt lần lượt là các nhúng của h, r và t). Nghĩa là, điểm TransE kvh + vr - vtkp của bộ ba hợp lệ (h, r, t) phải gần bằng 0 và nhỏ hơn điểm số kvh0 + vr 0 - vt 0kp của bộ ba không hợp lệ (h ', r', t '). Đặc tính chuyển tiếp trong TransE cũng bao hàm các mối quan hệ toàn cầu giữa các mục cùng chiều của vh, vr và vt.

Các mô hình dựa trên quá trình chuyển đổi khác mở rộng TransE để sử dụng bổ sung các vectơ hoặc ma trận chiếu để dịch các phần nhúng đầu và đuôi vào không gian vectơ quan hệ, chẳng hạn như: TransH (Wang et al., 2014), TransR (Lin et al., 2015b), TransD (Ji et al., 2015), STransE (Nguyen et al., 2016b) và TranSparse (Ji et al., 2016). Hơn nữa, DISTMULT (Yang et al., 2015) và ComplEx (Trouillon et al., 2016) sử dụng sản phẩm chấm ba tuyến tính để tính điểm cho mỗi bộ ba. Nghiên cứu gần đây đã chỉ ra rằng việc sử dụng các đường dẫn quan hệ giữa các thực thể trong KB có thể giúp có được thông tin theo ngữ cảnh để cải thiện hiệu suất hoàn thành KB (Lin et al., 2015a; Luo et al., 2015; Guu et al., 2015; Toutanova et al. , 2016; Nguyễn và cộng sự, 2016a). Xem các mô hình nhúng khác để hoàn thành KB trong Nguyễn (2017).

Gần đây, mạng nơ ron tích chập (CNNs), ban đầu được thiết kế cho thị giác máy tính (LeCun et al., 1998), đã nhận được sự chú ý nghiên cứu đáng kể trong xử lý ngôn ngữ tự nhiên (Collobert et al., 2011; Kim, 2014). CNN học các tính năng phi tuyến tính để nắm bắt các mối quan hệ phức tạp với số lượng tham số ít hơn đáng kể so với các mạng thần kinh được kết nối đầy đủ. Lấy cảm hứng từ sự thành công trong tầm nhìn máy tính, Dettmers et al. (2018) đề xuất ConvE Hồi mô hình đầu tiên áp dụng CNN cho nhiệm vụ hoàn thành KB. Trong ConvE, chỉ vh và vr được định hình lại và sau đó được nối vào một ma trận đầu vào được đưa vào lớp chập. Các bộ lọc khác nhau có cùng hình dạng 3 × 3 được vận hành trên ma trận đầu vào cho các thang đo bản đồ tính năng đầu ra. Các tenxơ bản đồ đặc trưng này sau đó được vector hóa và ánh xạ thành một vectơ thông qua một phép biến đổi tuyến tính. Sau đó, vectơ này được tính toán với vt thông qua một sản phẩm chấm để trả về điểm cho (h, r, t). Xem định nghĩa chính thức của hàm điểm ConvE trong Bảng 1. Điều đáng chú ý là ConvE tập trung vào các mối quan hệ cục bộ giữa các mục chiều khác nhau trong mỗi vh hoặc vr, nghĩa là, ConvE không quan sát các mối quan hệ toàn cầu giữa các mục cùng chiều của một nhúng bộ ba (vh, vr, vt), để ConvE bỏ qua đặc tính chuyển tiếp trong các mô hình dựa trên quá trình chuyển đổi, đây là một trong những trực giác hữu ích nhất cho tác vụ.

Trong bài báo này, chúng tôi trình bày ConvKB một mô hình nhúng trong đó đề xuất sử dụng CNN mới cho nhiệm vụ hoàn thành KB. Trong ConvKB, mỗi thực thể hoặc quan hệ được liên kết với một nhúng kdimensional duy nhất. Gọi vh, vr và vt lần lượt là các nhúng nhúng k chiều của h, r và t. Đối với mỗi bộ ba (h, r, t), bộ ba tương ứng của các chiều nhúng k (vh, vr, vt) được biểu diễn dưới dạng ma trận đầu vào k × 3. Ma trận đầu vào này được đưa đến lớp chập trong đó các bộ lọc khác nhau có cùng hình dạng 1 × 3 được sử dụng để trích xuất các mối quan hệ toàn cầu giữa các mục nhập cùng chiều của bộ ba nhúng. Nghĩa là, các bộ lọc này được vận hành lặp đi lặp lại trên mỗi hàng của ma trận đầu vào để tạo ra các bản đồ tính năng khác nhau. Các bản đồ đặc trưng được nối vào một vectơ đặc trưng sau đó được tính toán với một vectơ trọng lượng thông qua một sản phẩm chấm để tạo ra điểm số cho bộ ba (h, r, t). Điểm này được sử dụng để suy ra liệu bộ ba (h, r, t) có hợp lệ hay không.

* Chúng tôi giới thiệu ConvKB một mô hình nhúng mới lạ về các thực thể và các mối quan hệ để hoàn thành cơ sở tri thức. ConvKB mô hình các mối quan hệ giữa các mục cùng chiều của các nhúng. Điều này ngụ ý rằng ConvKB khái quát các đặc tính chuyển tiếp trong các mô hình nhúng dựa trên quá trình chuyển đổi.
* Chúng tôi đánh giá ConvKB trên hai bộ dữ liệu điểm chuẩn: WN18RR (Dettmers et al., 2018) và FB15k-237 (Toutanova và Chen, 2015). Kết quả thử nghiệm cho thấy ConvKB có được hiệu suất dự đoán liên kết tốt hơn so với các mô hình nhúng SOTA trước đây. Cụ thể, ConvKB có được thứ hạng trung bình tốt nhất và Số lần truy cập cao nhất @ 10 trên WN18RR và tạo ra thứ hạng đối ứng trung bình cao nhất và Số lần truy cập cao nhất @ 10 trên FB15k-237

## 2 Proposed ConvKB model

## 3 Experimen
### 3.1 Datase